In [ ]:
import torch
import torchvision
import cv2
import numpy as np
import math
from time import sleep
from helper_function import *
from helper_movement import *


In [ ]:
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

model.load_state_dict(torch.load('best_model.pth'))

device = torch.device('cuda')
model = model.to(device)

In [ ]:
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)
display(widgets.HBox([image, blocked_slider]))

In [ ]:
from jetbot import Robot

robot = Robot()
robot.stop()

curr_x = 0          #use to keep track on the x of the car current status
curr_y = 0          #use to keep track on the y of the car current status
curr_angle = 0      #use to keep track on the angle of the car current status, range from -180 to 180. 0 indicate straight
des_x = 4
des_y = 4
spin_around_time = 3.1
ended = False

In [ ]:
import torch.nn.functional as F
import time
import random


no_obj_go_time = 0.001
obj_spin_time_max = 1
obj_go_time = 2
go_forward_ct = 10000000
go_forward_before_redirect_max = 200 # #* 0.001s
start_time = None
last_change = -1 # 0 is left, 1 is forward, 2 is right

def update(change):
    global blocked_slider, robot, curr_x, curr_y, curr_angle 
    global des_x, des_y, spin_time
    global go_forward_ct, start_time, last_change, ended 
    
    if ended:
        return
    
    #if not des_x or not des_y:
    #    print("hahaha")
    
    if reach_destination(curr_x, des_x, curr_y, des_y):
        #camera.unobserve(update, names='value')
        robot.stop()
        print("stop")
        camera.unobserve(update, names='value')
        robot.stop()
        ended = True
        return
        
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    print("current location", curr_x, curr_y, curr_angle )
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    # takes only 0.05s
    '''
    if last_change == 0:
        # print("time", time.time()-start_time)
        curr_angle = check_angle(curr_angle + (360 * (time.time()-start_time)/ spin_around_time) )  #make sure the angle is between -180 to 180 degree
    elif last_change == 1:
        # print("time", time.time()-start_time)
        curr_x, curr_y = update_info_straight(curr_x,curr_y,curr_angle, (time.time()-start_time)) 
    '''
    
    
    if prob_blocked < 0.5: #if no object
        if start_time:
            curr_x, curr_y = update_info_straight(curr_x,curr_y,curr_angle, time.time() - start_time) 
        start_time = None
        target_angle = convert_angle(des_x - curr_x, des_y - curr_y)
        print("target", target_angle)
        angle_diff = angle_range_convert(target_angle - curr_angle) 
        print("angle diff",angle_diff)
        #if go_forward_ct > random.randint(6,9) * 0.1 * go_forward_before_redirect_max and abs(angle_diff) > 2: 
        if  abs(angle_diff) > 2:
            
                #not pointing at the correct angle
                #adjust to the left while going straight if angle is negative and right if angle is positive
            #straight()
            #...
            if angle_diff < 0: # target < current
                spin_right()
            else:
                spin_left()

            time_to_turn = spin_around_time * abs(angle_diff) / 360     #calculate how long does the spin need to be
            print("time_to_turn",time_to_turn)
            #time_to_turn = 0.001
            sleep(time_to_turn)     #sleep to wait until the car turn enough
            robot.stop()
            
            #curr_angle = target_angle    #x and y did not change, only the angle change to the target value in this case.
            
            #print("current location", curr_x, curr_y, curr_angle )
            
            if angle_diff > 0 :
                curr_angle = check_angle(curr_angle + (360 * time_to_turn / spin_around_time) )
            else:
                curr_angle = check_angle(curr_angle - (360 * time_to_turn / spin_around_time) )
            
        else:
            
            #go straight 
            straight()
            start_time = time.time()
            #sleep(no_obj_go_time)
            sleep(0.001)
            # curr_x, curr_y = update_info_straight(curr_x,curr_y,curr_angle, no_obj_go_time)   
            
            #print("current location", curr_x, curr_y, curr_angle )

            
    else:
        if start_time:
            curr_x, curr_y = update_info_straight(curr_x,curr_y,curr_angle, time.time() - start_time) 
        start_time = None
        spin_left()
        obj_spin_time = 0.7
        sleep(obj_spin_time)
        robot.stop()
        #obj_spin_time = 0.001
        curr_angle = check_angle(curr_angle + (360 * obj_spin_time / spin_around_time) )  #make sure the angle is between -180 to 180 degree
      
        straight()
        obj_go_time = 2
        sleep(obj_go_time)
        robot.stop()
        curr_x, curr_y = update_info_straight(curr_x, curr_y,curr_angle,obj_go_time)
        #curr_angle = check_angle(curr_angle + (360 * tmp/ spin_around_time) )
        #curr_x, curr_y = update_info_straight(curr_x, curr_y,curr_angle,obj_go_time)
        #print("current location", curr_x, curr_y, curr_angle )
        
    # time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize
camera.observe(update, names='value')

In [ ]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

In [ ]:
camera.unobserve(update, names='value')
robot.stop()

In [ ]:
robot.stop()

In [ ]:
def funcc(x, y):
    x = x + 1
    y = y + 1
    return x,y